<a href="https://colab.research.google.com/github/XiangChen0/Perturbation-treatment-for-strong-magnetic-field-instability/blob/main/MIT_6_036_Spring_2021_HW06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MIT 6.036 Spring 2021: Homework 06#

This colab notebook provides code and a framework for [Homework 06](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2021_Spring/courseware/Week6/week6_homework/).  You can work out your solutions here, then submit your results back on the homework page when ready.

**Note**: You can go to `File > Save a copy in Drive...` to save your own copy of this notebook for editing.

## <section>**Setup**</section>

First, download the code distribution for this homework that contains test cases and helper functions.

Run the next code block to download and import the code for this lab.


In [1]:
!rm -rf code_for_hw6*
!rm -rf mnist
!rm -rf data
!wget --no-check-certificate --quiet https://introml.odl.mit.edu/cat-soop/_static/6.036/homework/hw06/code_for_hw6.zip
!unzip code_for_hw6.zip
!mv code_for_hw6/* .

from code_for_hw6 import *
import numpy as np
import modules_disp as disp


Archive:  code_for_hw6.zip
   creating: code_for_hw6/
  inflating: code_for_hw6/.DS_Store  
  inflating: __MACOSX/code_for_hw6/._.DS_Store  
  inflating: code_for_hw6/modules_disp.py  
  inflating: code_for_hw6/utils_hw6.py  
  inflating: code_for_hw6/code_for_hw6.py  
   creating: code_for_hw6/data/
  inflating: code_for_hw6/expected_results.py  
  inflating: code_for_hw6/data/data5_train.csv  
  inflating: code_for_hw6/data/data3_train.csv  
  inflating: code_for_hw6/data/data4_train.csv  
  inflating: code_for_hw6/data/data4_validate.csv  
  inflating: code_for_hw6/data/data3_validate.csv  
  inflating: code_for_hw6/data/dataXor_train.csv  
  inflating: code_for_hw6/data/data2_train.csv  
  inflating: code_for_hw6/data/data2_validate.csv  
  inflating: code_for_hw6/data/data5_validate.csv  
  inflating: code_for_hw6/data/data3class_train.csv  
  inflating: code_for_hw6/data/data1_validate.csv  
  inflating: code_for_hw6/data/data1_train.csv  


# Implementing Neural Networks

This problem follows directly from the **Backpropagation** problem in [Homework 05](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2021_Spring/courseware/Week5/week5_homework/). Make sure that you have done and understood that problem before beginning this problem.

Although for "real" applications you want to use one of the many packaged implementations of neural networks (we'll start using one of those soon), there is no substitute for implementing one yourself to get an in-depth understanding. Luckily, that is relatively easy to do if we're not too concerned with maximum efficiency.

We'll use the modular implementation that we guided you through in the previous problem, which leads to clean code. The basic framework for SGD training is given below. We can construct a network and train it as follows:

```
# build a 3-layer network
net = Sequential([Linear(2,3), Tanh(),
                  Linear(3,3), Tanh(),
    	          Linear(3,2), SoftMax()])
# train the network on data and labels
net.sgd(X, Y)
```
We call this a Sequential object because we feed the outputs of the previous layer into the next, you could say this forms a sequential relationship between layers.

We will (later) be generalizing SGD to operate on a "mini-batch" of data points instead of a single point. You should strive for an implementation of the forward, backward, and `class_fun` methods that works with batches of data. Note that when $b$ is mentioned as part of the shape of a matrix in the code, this $b$ refers to the number of points.

## 1) Linear Modules: ##
Each linear module has a forward method that takes in a batch of activations $A$ (from the previous layer) and returns a batch of pre-activations $Z$.

Each linear module has a backward method that takes in `dLdZ` and returns `dLdA`. This module also computes and stores `dLdW` and `dLdW0`, the gradients with respect to the weights.

Hint: be careful with dimensions when computing `dLdW0`. `dLdZ` is $(n \times b)$, but `dLdW0` is $(n \times 1)$. Why do you need to sum over all $b$ points in the batch when computing `dLdW0`?

In [2]:
class Module:
    def sgd_step(self, lrate): pass  # For modules w/o weights

class Linear(Module):
    def __init__(self, m, n):
        # initializes the weights randomly and offsets as 0
        self.m, self.n = (m, n)  # (in size, out size)
        self.W0 = np.zeros([self.n, 1])  # (n x 1)
        self.W = np.random.normal(0, 1.0 * m ** (-.5), [m, n])  # (m x n)

    def forward(self, A):
        # store the input matrix for future use
        self.A = A   # (m x b)  Hint: make sure you understand what b stands for
        return np.dot(self.W.T, A) + self.W0  # Your code (n x b)

    def backward(self, dLdZ):  
        # dLdZ is (n x b), uses stored self.A
        # store the derivatives for use in sgd_step and return dLdA
        self.dLdW = np.dot(self.A, dLdZ.T)       # Your code
        self.dLdW0 = np.dot(dLdZ, np.ones([dLdZ.shape[1],1]))     # Your code
        return np.dot(self.W, dLdZ)        # Your code: return dLdA (m x b)

    def sgd_step(self, lrate):  # Gradient descent step
        self.W = self.W - lrate * self.dLdW          # Your code
        self.W0 = self.W0 - lrate * self.dLdW0       # Your code
    

 You are **highly encouraged** to make your own tests for each module. The test cases being run on catsoop are given below for your reference:

In [ ]:
def nn_linear_forward():
    np.random.seed(0)
    linear = Linear(2,3)
    X,Y = super_simple_separable()
    return linear.forward(X).tolist()

expected_linear_forward = [[10.417500637754383, 6.911221682745654, 20.733665048236965, 22.891234399772113], [7.168722346625092, 3.489987464919749, 10.469962394759248, 9.998261102396512], [-2.071054548689073, 0.6941371647696142, 2.0824114943088414, 4.849668106971125]]
print("Pass linear forward? ", np.allclose(nn_linear_forward(), expected_linear_forward))

def nn_linear_forward_bias():
    np.random.seed(0)
    linear = Linear(2,3)
    linear.W0 = np.array([[1],[1],[1]])
    X,Y = super_simple_separable()
    return linear.forward(X).tolist()

expected_linear_forward_bias = [[11.417500637754383, 7.911221682745654, 21.733665048236965, 23.891234399772113], [8.168722346625092, 4.489987464919749, 11.469962394759248, 10.998261102396512], [-1.071054548689073, 1.6941371647696142, 3.0824114943088414, 5.849668106971125]]
print("Pass linear forward bias? ", np.allclose(nn_linear_forward_bias(), expected_linear_forward_bias))



In [ ]:
def nn_linear_backward():
    np.random.seed(0)
    linear = Linear(2,3)
    X,Y = super_simple_separable()
    linear.forward(X)
    dLdZ = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0],
                                     [3, 0, 0, 1]])
    return linear.backward(dLdZ).tolist()

expected_linear_backward = [[3.889497924054116, 1.247373376201773, 0.2829538755771419, 0.6920722655660196], [2.1525571673658237, 1.5845507770701677, 1.3205629190941617, -0.6910398159642225]]
print("Pass linear backward? ", np.allclose(nn_linear_backward(), expected_linear_backward))

def nn_linear_backward_stored_dLdW_dLdW0():
    np.random.seed(0)
    linear = Linear(2,3)
    X,Y = super_simple_separable()
    linear.forward(X)
    dLdZ = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0],
                                     [3, 0, 0, 1]])
    linear.backward(dLdZ)
    return [linear.dLdW.tolist(), linear.dLdW0.tolist()]

expected_linear_backward_stored = [[[5, 13, 18], [7, 16, 20]], [[2], [3], [4]]]
print("Pass linear backward stored vals dLdW? ", np.allclose(nn_linear_backward_stored_dLdW_dLdW0()[0], expected_linear_backward_stored[0]))
print("Pass linear backward stored vals dLdW0? ", np.allclose(nn_linear_backward_stored_dLdW_dLdW0()[1], expected_linear_backward_stored[1]))

In [ ]:
def nn_linear_sgd():
    np.random.seed(0)
    linear = Linear(2,3)
    X,Y = super_simple_separable()
    linear.forward(X)
    dLdZ = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0],
                                     [3, 0, 0, 1]])
    linear.backward(dLdZ)
    linear.sgd_step(0.005)
    return [np.vstack([linear.W, linear.W0.T]).tolist()]

expected_linear_sgd = [[[1.222373376201773, 0.2179538755771419, 0.6020722655660197], [1.5495507770701678, 1.2405629190941616, -0.7910398159642225], [-0.01, -0.015, -0.02]]]
print("Pass linear sgd? ", np.allclose(nn_linear_sgd(), expected_linear_sgd))

The following datasets are defined for your use:
*  `super_simple_separable_through_origin()`
*  `super_simple_separable()`
*  `xor()`
*  `xor_more()`
*  `hard()`

Further, a plotting function is defined for your usage in modules_disp.py, and can be called in the colab notebook as `disp.plot_nn()`.
```
def plot_nn(X, Y, nn):
    """ Plot output of nn vs. data """
    def predict(x):
        return nn.modules[-1].class_fun(nn.forward(x))[0]
    xmin, ymin = np.min(X, axis=1)-1
    xmax, ymax = np.max(X, axis=1)+1
    nax = plot_objective_2d(lambda x: predict(x), xmin, xmax, ymin, ymax)
    plot_data(X, Y, nax)
    plt.show()```


## 2) Activation functions: ##
Each activation module has a forward method that takes in a batch of pre-activations $Z$ and returns a batch of activations $A$. (Ask yourself, why is this the reverse of the linear module?)

Each activation module has a backward method that takes in `dLdA` and returns `dLdZ`, with the exception of SoftMax, where we assume `dLdZ` is passed in.

### 2.1) Tanh: ###
Please use `np.tanh` here. You can find documentation online.

Hint 1: the derivative of $\tanh$ is given by $\frac{d\tanh(z)}{d z} = 1 - \tanh(z)^2$. 

Hint 2: If you're having problems with this, try writing out the shapes of everything involved and remember that `@` does matrix multiplication while `*` does component-wise multiplication.

In [3]:
class Tanh(Module):            # Layer activation
    def forward(self, Z):
        self.A = np.tanh(Z)            # Your code
        return self.A

    def backward(self, dLdA):    # Uses stored self.A
        return dLdA*(1-self.A*self.A)            # Your code: return dLdZ

Tanh unit test:

In [ ]:
def nn_tanh_forward():
    np.random.seed(0)
    tanh = Tanh()
    X,Y = super_simple_separable()
    return tanh.forward(X).tolist()

expected_tanh_forward = [[0.9640275800758169, 0.9950547536867305, 0.999999969540041, 0.9999999999244973], [0.9999092042625951, 0.9640275800758169, 0.9999877116507956, 0.9999092042625951]]
print("Pass tanh forward? ", np.allclose(nn_tanh_forward(), expected_tanh_forward))

def nn_tanh_backward():
    np.random.seed(0)
    tanh = Tanh()
    X,Y = super_simple_separable()
    tanh.forward(X)
    dLdA = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0]])
    return tanh.backward(dLdA).tolist()

expected_tanh_backward = [[0.07065082485316443, 0.009866037165440211, 0.0, 0.0], [0.0003631664618877206, 0.0, 2.4576547405286142e-05, 0.0]]
print("Pass tanh backward? ", np.allclose(nn_tanh_backward(), expected_tanh_backward))

###2.2) ReLU: ###
Hint:
[`np.maximum`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.maximum.html) might be useful.

In [4]:
class ReLU(Module):              # Layer activation
    def forward(self, Z):
        self.A = np.maximum(Z,np.zeros(Z.shape))            # Your code
        return self.A

    def backward(self, dLdA):    # uses stored self.A
        return dLdA * np.sign(self.A)         # Your code: return dLdZ

ReLU unit tests:

In [ ]:
def nn_relu_forward():
    np.random.seed(0)
    relu = ReLU()
    X,Y = super_simple_separable()
    return relu.forward(X).tolist()

expected_relu_forward = [[2, 3, 9, 12], [5, 2, 6, 5]]
print("Pass relu forward? ", np.allclose(nn_relu_forward(), expected_relu_forward))

def nn_relu_backward():
    np.random.seed(0)
    relu = ReLU()
    X,Y = super_simple_separable()
    relu.forward(X)
    dLdA = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0]])
    return relu.backward(dLdA).tolist()

expected_relu_backward = [[1, 1, 0, 0], [2, 0, 1, 0]]
print("Pass relu backward? ", np.allclose(nn_relu_backward(), expected_relu_backward))


###2.3) SoftMax: ###
We will now implement the softmax activation function. Note that we will assume that the derivative `dLdZ` is passed into the backward method. We also have the additional `class_fun` method:

* `SoftMax.class_fun`: Given `Ypred`, a matrix where each column corresponds to a data point's vector of class probabilities (computed by Softmax), this returns a vector of the classes (integers) with the highest probability for each point. The vector should be 1-dimensional with shape `(b,)`.

In [5]:
class SoftMax(Module):           # Output activation
    def forward(self, Z):
        return np.exp(Z)/np.sum(np.exp(Z),axis=0)              # Your code

    def backward(self, dLdZ):    # Assume that dLdZ is passed in
        return dLdZ

    def class_fun(self, Ypred):  
        # Returns the index of the most likely class for each point as vector of shape (b,)
        return np.argmax(Ypred, axis=0)              # Your code
    

Test cases for softmax:

In [ ]:
def nn_softmax_forward():
    np.random.seed(0)
    softmax = SoftMax()
    X,Y = super_simple_separable()
    return softmax.forward(X).tolist()

expected_softmax_forward = [[0.04742587317756679, 0.7310585786300048, 0.9525741268224334, 0.9990889488055993], [0.9525741268224333, 0.2689414213699951, 0.04742587317756678, 0.0009110511944006454]]
print("Pass softmax forward? ", np.allclose(nn_softmax_forward(), expected_softmax_forward))
  
def nn_softmax_class_fun():
    np.random.seed(0)
    softmax = SoftMax()
    X,Y = super_simple_separable()
    Ypred = softmax.forward(X)
    return softmax.class_fun(Ypred).tolist()

expected_softmax_class_fun = [1, 0, 0, 0]
print("Pass softmax class fun? ", np.allclose(nn_softmax_class_fun(), expected_softmax_class_fun))

## 3) Loss Functions:##
We will now implement the NLL loss function assuming that the output activation function is softmax and using the definition from homework 5:

$$NLL(y^{pred}, y) = -\sum_{j = 1}^{n^L} y_j \ln y^{pred}_j.$$

Hint 1: since we are using Softmax $ = SM(Z)$ at the output layer and
 _NLL_ as the $Loss(A, Y)$ function, we have seen that there is a
simple form for ${\tt dLdZ} = \frac{\partial Loss}{\partial Z}$;
namely, it is the prediction error $Y^{pred} - Y$ (check HW 6 for a refresher).  A similar result holds
when using _NLL_ with a sigmoid output activation or a quadratic loss
with a linear output activation.

Hint 2: Note that `np.log` might be helpful. You can find documentation online.

In [6]:
class NLL(Module):       # Loss
    def forward(self, Ypred, Y):
        # returns loss as a float
        self.Ypred = Ypred
        self.Y = Y
        return -np.sum(Y*np.log(Ypred))      # Your code

    def backward(self):  # Use stored self.Ypred, self.Y
        # note, this is the derivative of loss with respect to the input of softmax
        return self.Ypred-self.Y     # Your code
    

NLL Test Cases:

In [ ]:
def nn_nll_forward():
    nll = NLL()
    y = np.array([[1, 0, 1, 0]])
    Y = for_softmax(y)
    ypred = np.array([[.7, .3, .99, .99]])
    Ypred = for_softmax(ypred)
    return nll.forward(Ypred, Y)

expected_nll_forward = 5.328570409719057
print("Pass nll forward? ", np.allclose(nn_nll_forward(), expected_nll_forward))

def nn_nll_backward():
    nll = NLL()
    y = np.array([[1, 0, 1, 0]])
    Y = for_softmax(y)
    ypred = np.array([[.7, .3, .99, .99]])
    Ypred = for_softmax(ypred)
    nll.forward(Ypred, Y)
    return nll.backward().tolist()

expected_nll_backward = [[0.30000000000000004, -0.30000000000000004, 0.010000000000000009, -0.99], [-0.30000000000000004, 0.3, -0.010000000000000009, 0.99]]
print("Pass nll backward? ", np.allclose(nn_nll_backward(), expected_nll_backward))

## Activation and Loss Test Cases: ##
Run Test 1 and Test 2 below and compare your outputs with the expected outputs.


In [ ]:
# TEST 1: sgd_test for Tanh activation and SoftMax output
np.random.seed(0)
sgd_test(Sequential([Linear(2,3), Tanh(), Linear(3,2), SoftMax()], NLL()), test_1_values)

In [ ]:
# TEST 2: sgd_test for ReLU activation and SoftMax output
np.random.seed(0)
sgd_test(Sequential([Linear(2,3), ReLU(), Linear(3,2), SoftMax()], NLL()), test_2_values)

## 4) Sequential ##

Now we will implement SGD following these specs:

* Randomly pick a data point `Xt`, `Yt` by using `np.random.randint` to choose a random index into the data.
Compute the predicted output `Ypred` for `Xt` with the forward method.
* Compute the loss for `Ypred` relative to `Yt`.
* Use the backward method to compute the gradients. (Hint: where should backprop start? Which module has a backward that takes no input?)
* Use the `sgd_step` method to change the weights.
* Repeat.

For context, we can construct a network and train it as follows:

```
# build a 3-layer network
net = Sequential([Linear(2,3), Tanh(),
                  Linear(3,3), Tanh(),
                  Linear(3,2), SoftMax()], NLL())
# train the network on data and labels
net.sgd(X, Y)
```


In [7]:
class Sequential:
    def __init__(self, modules, loss):            # List of modules, loss module
        self.modules = modules
        self.loss = loss

    def sgd(self, X, Y, iters=100, lrate=0.005):  # Train
        D, N = X.shape
        for it in range(iters):
            rdn = np.random.randint(N) # YOUR CODE HERE
            Xt = X[:,rdn:rdn+1]
            Yt = Y[:,rdn:rdn+1]
            Ypred = self.forward(Xt)
            cur_loss = self.loss.forward(Ypred, Yt)
            self.print_accuracy(it, X, Y, cur_loss, every=1)
            delta = self.loss.backward()
            self.backward(delta)
            self.sgd_step(lrate)
            
    def forward(self, Xt):                        # Compute Ypred
        for m in self.modules: Xt = m.forward(Xt)
        return Xt

    def backward(self, delta):                    # Update dLdW and dLdW0
        # Note reversed list of modules
        for m in self.modules[::-1]: delta = m.backward(delta)

    def sgd_step(self, lrate):                    # Gradient descent step
        for m in self.modules: m.sgd_step(lrate)

    def print_accuracy(self, it, X, Y, cur_loss, every=250):
        # Utility method to print accuracy on full dataset, should
        # improve over time when doing SGD. Also prints current loss,
        # which should decrease over time. Call this on each iteration
        # of SGD!
        if it % every == 1:
            cf = self.modules[-1].class_fun
            acc = np.mean(cf(self.forward(X)) == cf(Y))
            print('Iteration =', it, '	Acc =', acc, '	Loss =', cur_loss)
            



Use Test 3 to help you debug.

In [ ]:
# TEST 3: try calling these methods that train with a simple dataset
from pprint import pprint

tanh_expected = [np.array([[1.2473733761848262, 0.2829538808226157, 0.6924193292712828],
                           [1.5845507770278007, 1.320562932207846, -0.6901721567010647],
                           [-8.47337764291184e-12, 2.6227368810847102e-09, 0.00017353185263155828]]),
                 np.array([[0.544808855557535, -0.08366117689965663],
                           [-0.06331837550937103, 0.24078409926389266],
                           [0.08677202043839037, 0.8360167748667923],
                           [-0.0037249480614717995, 0.0037249480614718]])]

relu_expected = [np.array( [[1.2421914999646917, 0.2851239946607419, 0.6905003767490479],
                            [1.5695659964519526, 1.3273884281993562, -0.6920877418422037],
                            [-0.002775491757223511, 0.001212351486908601, -0.0005239629389906042]]),
                 np.array([[0.501769700845158, -0.04062202218727964],
                           [-0.09260786974986725, 0.27007359350438886],
                           [0.08364438851530624, 0.8391444067898763],
                           [-0.004252310922204505, 0.004252310922204505]])]

pred_expected = [np.array([0, 0, 0, 0]),
                 np.array([8.565750618357672])]

def nn_tanh_test():
    np.random.seed(0)
    nn = Sequential([Linear(2, 3), Tanh(), Linear(3, 2), SoftMax()], NLL())
    X, Y = super_simple_separable()
    nn.sgd(X, Y, iters=1, lrate=0.005)
    out = [np.vstack([nn.modules[0].W, nn.modules[0].W0.T]).tolist(),
            np.vstack([nn.modules[2].W, nn.modules[2].W0.T]).tolist()]
    print("Output was:")
    pprint(out)
    print("Passed?", np.allclose(out[0], tanh_expected[0]) and np.allclose(out[1], tanh_expected[1]))    


def nn_relu_test():
    np.random.seed(0)
    nn = Sequential([Linear(2, 3), ReLU(), Linear(3, 2), SoftMax()], NLL())
    X, Y = super_simple_separable()
    nn.sgd(X, Y, iters=2, lrate=0.005)
    out = [np.vstack([nn.modules[0].W, nn.modules[0].W0.T]).tolist(),
            np.vstack([nn.modules[2].W, nn.modules[2].W0.T]).tolist()]
    print("Output was:")
    pprint(out)
    print("Passed?", np.allclose(out[0], relu_expected[0]) and np.allclose(out[1], relu_expected[1]))


def nn_pred_test():
    np.random.seed(0)
    nn = Sequential([Linear(2, 3), ReLU(), Linear(3, 2), SoftMax()], NLL())
    X, Y = super_simple_separable()
    nn.sgd(X, Y, iters=1, lrate=0.005)
    Ypred = nn.forward(X)
    out = nn.modules[-1].class_fun(Ypred).tolist(), [nn.loss.forward(Ypred, Y)]
    print("Output was:")
    pprint(out)
    print("Passed?", np.allclose(out[0], pred_expected[0]) and np.allclose(out[1], pred_expected[1]))


In [ ]:
nn_tanh_test()

# Expected output:
# '''
# [[[1.2473733761848262, 0.2829538808226157, 0.6924193292712828],
#   [1.5845507770278007, 1.320562932207846, -0.6901721567010647],
#   [-8.47337764291184e-12, 2.6227368810847106e-09, 0.00017353185263155828]],
#  [[0.544808855557535, -0.08366117689965663],
#   [-0.06331837550937104, 0.24078409926389266],
#   [0.08677202043839037, 0.8360167748667923],
#   [-0.0037249480614718, 0.0037249480614718]]]
# '''

In [ ]:
nn_relu_test()

# Expected output:
# '''
# [[[1.2421914999646917, 0.2851239946607419, 0.6905003767490479],
#   [1.5695659964519526, 1.3273884281993562, -0.6920877418422037],
#   [-0.0027754917572235106, 0.001212351486908601, -0.0005239629389906042]],
#  [[0.501769700845158, -0.040622022187279644],
#   [-0.09260786974986723, 0.27007359350438886],
#   [0.08364438851530624, 0.8391444067898763],
#   [-0.004252310922204504, 0.004252310922204505]]]
# '''

In [ ]:
nn_pred_test()

# Expected output:
# '''
# ([0, 0, 0, 0], [8.56575061835767])
# '''

In [15]:
class Sigmoid(Module):            # Layer activation
    def forward(self, Z):
        self.A = 1/(1+np.exp(-Z))            # Your code
        return self.A

    def backward(self, dLdA):    # Uses stored self.A
        return dLdA*self.A*(1-self.A)            # Your code: return dLdZ

class Quad(Module):            # Layer activation
    def forward(self, Z):
        self.A = Z*Z 
        self.Z = Z        # Your code
        return self.A

    def backward(self, dLdA):    # Uses stored self.A
        return dLdA* 2 * self.Z

class MSE(Module):       # Loss
    def forward(self, Ypred, Y):
        # returns loss as a float
        self.Ypred = Ypred
        self.Y = Y
        return np.sum((Ypred-Y)*(Ypred-Y))     # Your code

    def backward(self):  # Use stored self.Ypred, self.Y
        # note, this is the derivative of loss with respect to the input of softmax
        return 2*(self.Ypred-self.Y)    # Your code

net = Sequential([Linear(2,4), ReLU(), Linear(4,1), Sigmoid()], MSE())
# train the network on data and labels
X= np.array([[-1.5,1.5],[0,1.5],[1.5,1.5],[1.5,0],[1.5,-1.5],[0,-1.5],[-1.5,-1.5],[-1.5,0],[-0.5,-0.5],[-0.5,0.5],[0.5,0.5],[0.5,-0.5]]).T
Y = np.array([[1],[1],[1],[1],[1],[1],[1],[1],[0],[0],[0],[0]]).T
net.sgd(X, Y, iters=100000, lrate=0.1)
Ypred = net.forward(X)
print(Ypred)



[[1.         0.9903569  0.99999999 0.99061392 0.9964275  0.9964511
  0.99647453 0.99034348 0.00571494 0.01522654 0.01514916 0.00568981]]


In [14]:
print(net.modules[0].W)
print(net.modules[0].W0)
print(net.modules[2].W)
print(net.modules[2].W0)

[[ 2.19300728  2.34246014 -2.68816739  0.11681732]
 [ 2.82429688  0.20660751  2.83751282  2.28589481]]
[[-0.31557096]
 [-1.06794027]
 [-0.23632433]
 [ 3.42894123]]
[[ 3.53396985]
 [ 2.39532171]
 [ 3.67909773]
 [-4.0472499 ]]
[[4.21657051]]


In [16]:
net = Sequential([Linear(2,2), Quad(), Linear(2,1), Sigmoid()], MSE())
# train the network on data and labels
X= np.array([[-1.5,1.5],[0,1.5],[1.5,1.5],[1.5,0],[1.5,-1.5],[0,-1.5],[-1.5,-1.5],[-1.5,0],[-0.5,-0.5],[-0.5,0.5],[0.5,0.5],[0.5,-0.5]]).T
Y = np.array([[1],[1],[1],[1],[1],[1],[1],[1],[0],[0],[0],[0]]).T
net.sgd(X, Y, iters=100000, lrate=0.1)
Ypred = net.forward(X)
print(Ypred)

[[0.99999997 0.99407165 0.99999997 0.99392864 0.99999997 0.99416746
  0.99999997 0.99399073 0.0128898  0.01276258 0.01277692 0.01278801]]


In [17]:
print(net.modules[0].W)
print(net.modules[0].W0)
print(net.modules[2].W)
print(net.modules[2].W0)

[[-0.86438876  1.87954685]
 [ 1.83603221  0.88808201]]
[[-0.00065058]
 [-0.00104702]]
[[1.31297007]
 [1.25110123]]
[[-7.04847998]]
